In [1]:
import pandas as pd
import requests
import os 
import json
import dotenv
import datetime
from datetime import datetime
import time


dotenv.load_dotenv()

/Users/ashish/files/fun_projects/fantasy_football_2024/venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


False

In [23]:
SLEEPER_ALL_PLAYERS_URL = "https://api.sleeper.app/v1/players/nfl"
ALL_PLAYERS_FILE = "data/sleeper/all_players.json"

def get_all_players():
    response = requests.get(SLEEPER_ALL_PLAYERS_URL)
    response.raise_for_status()
    return response.json()

def check_all_players_file():
    '''return true if the all players file exists from the current day, false otherwise'''
    if os.path.exists(ALL_PLAYERS_FILE):
        file_time = os.path.getmtime(ALL_PLAYERS_FILE)
        file_date = datetime.fromtimestamp(file_time)
        current_date = datetime.now()
        if file_date.date() == current_date.date():
            return True
    return False

if not check_all_players_file():
    print("Updating all players file")
    all_players = get_all_players()
    with open(ALL_PLAYERS_FILE, "w") as f:
        json.dump(all_players, f, indent=2)
else: 
    print("All players file is up to date")

with open(ALL_PLAYERS_FILE, "r") as f:
    data = json.load(f)

# Extract the keys and merge with the dictionaries
records = [{"id": key, **value} for key, value in data.items()]

# Convert to DataFrame
df = pd.DataFrame(records)

df.to_csv("../data/sleeper/all_players.csv", index=False)





All players file is up to date


In [50]:
print(df.columns)
print(df.shape)


# df = df[df["position"].isin(keep_pos)]
# print(df["position"].value_counts())
# print(df.shape)

# Filter criteria
positions_of_interest = ["WR", "RB", "TE", "QB", "K", "DEF", "DST"]
max_search_rank = 150

df = df[
    (df['active'] == True) &
    (df['position'].isin(positions_of_interest)) &
    (df['search_rank'] <= max_search_rank) 
]
df.shape




Index(['id', 'depth_chart_position', 'first_name', 'height', 'sport',
       'full_name', 'team_abbr', 'espn_id', 'practice_participation',
       'position', 'news_updated', 'competitions', 'rotoworld_id', 'swish_id',
       'injury_start_date', 'birth_date', 'fantasy_data_id', 'pandascore_id',
       'fantasy_positions', 'birth_city', 'yahoo_id', 'practice_description',
       'injury_body_part', 'birth_country', 'search_rank', 'rotowire_id',
       'search_first_name', 'status', 'search_full_name', 'last_name',
       'injury_status', 'player_id', 'hashtag', 'oddsjam_id', 'number',
       'depth_chart_order', 'years_exp', 'age', 'weight', 'active', 'opta_id',
       'birth_state', 'gsis_id', 'team', 'search_last_name', 'stats_id',
       'metadata', 'sportradar_id', 'high_school', 'college', 'injury_notes'],
      dtype='object')
(171, 51)


205      Javonte Williams
272            Drake Maye
275       Jonathan Taylor
289          J.K. Dobbins
432           Breece Hall
               ...       
10094    Matthew Stafford
10129        Dak Prescott
10386          Kyle Pitts
10505         Jerry Jeudy
10516         CeeDee Lamb
Name: full_name, Length: 171, dtype: object

In [48]:
12*13

156

In [22]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials

SLEEPER_ADP_SHEET = "https://docs.google.com/spreadsheets/d/1wmjxi3K5rjIYME_lskUvquLbN331YV0vi-kg5VakpdY"
gspread_key = "fantasy-football-2024-430622-70fe6adeafe5.json"
# Define the scope and credentials
scope = ['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive']
credentials = ServiceAccountCredentials.from_json_keyfile_name(gspread_key, scope)

# Authenticate and open the Google Sheet
gc = gspread.authorize(credentials)

sheet = gc.open_by_url(SLEEPER_ADP_SHEET)
vals = sheet.get_worksheet(1).get_all_values()
pd.DataFrame(vals[1:], columns=vals[0])


,Date,Redraft PPR ADP,Redraft SF ADP,Redraft Half PPR ADP,Dynasty PPR ADP,Dynasty SF ADP,Dynasty Half PPR ADP,IDP ADP,Player Team,Player First Name,Player Last Name,Fantasy Player Position,Player Id,Positional Rank
0,"July 12, 2024",37,3.5,35.5,17.6,1,17.3,2.1,KC,Patrick,Mahomes,QB,4046,QB1
1,"July 12, 2024",22,1.5,19.6,21.9,2.8,21.5,1.5,BUF,Josh,Allen,QB,4984,QB2
2,"July 12, 2024",41.4,5.5,44.9,23.4,3.2,24,6.2,HOU,C.J.,Stroud,QB,9758,QB3
3,"July 12, 2024",34.2,4.6,31.5,29.1,4,29.5,6.4,PHI,Jalen,Hurts,QB,6904,QB4
4,"July 12, 2024",39.7,7.5,37.4,34,5.2,35.9,9.2,BAL,Lamar,Jackson,QB,4881,QB5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
479,"July 12, 2024",,,,260.2,,236.9,,NE,JuJu,Smith-Schuster,WR,4040,WR129
480,"July 12, 2024",,,292.5,,,,,IND,Derek,Carrier,TE,1718,TE48
481,"July 12, 2024",,,295.8,,,,,SF,Tristan,Vizcaino,K,5823,K16
482,"July 12, 2024",,,296,,,,,IND,Aldrick,Rosas,K,3827,K17


In [7]:
import pandas as pd
import re
from datetime import datetime

# Step 1: Define the URL of the Google Sheet
sheet_id = "1wmjxi3K5rjIYME_lskUvquLbN331YV0vi-kg5VakpdY"  # Replace with your Google Sheet ID

# # Step 2: Get the list of sheets and parse them
# def get_sheet_names(sheet_id):
#     sheet_url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv"
#     df = pd.read_csv(sheet_url)
#     print(df)
#     sheets = df.columns
#     valid_sheets = [sheet for sheet in sheets if re.match(r'^[A-Za-z]+\s\d{1,2},\s\d{4}\sADP$', sheet)]
#     return valid_sheets

# # Step 3: Get the most recent sheet name
# valid_sheets = get_sheet_names(sheet_id)
# print(valid_sheets)
# valid_sheets.sort(key=lambda date: datetime.strptime(date, '%B %d, %Y ADP'), reverse=True)
# most_recent_sheet = valid_sheets[0]

# Step 4: Read the data from the most recent sheet
# sheet_url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={most_recent_sheet}"
# df = pd.read_csv(sheet_url)

# print(df)


Empty DataFrame
Columns: [Instructions: Copy and past the data into your own google sheet. You can then sort the data based on the ADP list that's relevant to you.

Please give credit to Sleeper by backlinking to the Sleeper website https://sleeper.app/ in any articles or posts and tagging our social media accounts. 

Notes:

Only 10-12 Team Leagues
Weighs more recent drafts higher than older drafts
Discards rookie only / vet only drafts
IDP data includes redraft and dynasty data, mostly Dynasty SF leagues (thus the high QB ADPs)
Currently, we're seeing pretty signifcant number of new Dynasty SF league startups compared to other league types.
Top 300 for Redraft PPR, Redraft SF, Dynasty PPR, Dynasty SF
Top 400 for IDP   
Sheet is currently sorted by Dynasty SF ADP                                                                                          ]
Index: []
[]


In [ ]:

url = "https://docs.google.com/spreadsheets/d/1wmjxi3K5rjIYME_lskUvquLbN331YV0vi-kg5VakpdY/export?format=csv"

In [35]:
import yaml
with open("config.yaml", "r") as f:
    config = yaml.safe_load(f)

df_players = pd.read_csv(config["files"]["sleeper"]["all_players_csv"])
df_players["id"] = df_players["id"].astype(str)


# read adp file
df_adp = pd.read_csv(config["files"]["sleeper"]["adp"])
# get last integer in positional_rank str column
df_adp["rank_int"] = df_adp["Positional Rank"].str.extract("(\d+)").astype(int)
# filter to only rank_int <= 20
df_adp = df_adp[df_adp["rank_int"] <= 20]


# filter df_players such that id is in df_adp["Player ID"]
# df_adp["Player Id"]
# df_players["id"]
df_players = df_players[df_players["id"].isin(df_adp["Player Id"].astype(str))]

print(df_players.shape)
print(df_players.position.value_counts())
df_players.to_csv("tmp.csv")

(183, 51)
position
WR    20
TE    20
QB    20
LB    20
RB    20
DB    20
K     18
DE    15
DL    12
DT    10
CB     8
Name: count, dtype: int64


In [46]:
import pandas as pd


url = "https://fantasydata.com/nfl/fantasy-football-leaders?scope=season&scoring=fpts_half_ppr&order_by=fpts_half_ppr&sort_dir=desc&sp={season}"
seasons = ["2024_PRE", "2023_POST", "2023_REG", "2023_PRE", "2022_POST", "2022_REG", "2022_PRE", "2021_POST", "2021_REG", "2021_PRE", "2020_POST", "2020_REG", "2019_POST", 
           "2019_REG", "2018_POST", "2018_REG", "2017_POST", "2017_REG", "2016_POST", "2016_REG", "2015_POST", "2015_REG", "2014_POST", "2014_REG", "2013_POST", "2013_REG",
              "2012_POST", "2012_REG", "2011_POST", "2011_REG", "2010_POST", "2010_REG", "2009_POST", "2009_REG", "2008_POST", "2008_REG", "2007_POST", "2007_REG", "2006_POST",
                "2006_REG", "2005_POST", "2005_REG", "2004_POST", "2004_REG", "2003_POST", "2003_REG", "2002_POST", "2002_REG", "2001_POST", "2001_REG"]

season = seasons[1]

tables = pd.read_html(url.format(season=seasons[1]), header=1)

df = tables[0]

cols = {
    "RK": "rank", "NAME": "name", "TEAM": "team", "POS": "position", "GP": "games_played", "FPTS": "fpts", "FPTS/G": "fpts_per_game", 
    "YDS": "pass_yds", "TD": "pass_td", "INT": "pass_int", "YDS.1": "rush_yds", "TD.1": "rush_td", "REC": "rec", "YDS.2": "rec_yds",
    "TD.2": "rec_td", "SCK": "sacks", "INT.1": "def_int", "FF": "fumbles_forced", "FR": "fumbles_rec"
}

df.rename(columns=cols, inplace=True)

df.to_csv(f"data/fantasydata/fpts_ssn_{season}.csv", index=False)